## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [5]:
classifier=Sequential()

#卷積組合
input_shape = (32, 32, 3)
classifier.add(Convolution2D(32, kernel_size=(3, 3), padding='same',input_shape=input_shape,activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())


'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())


#flatten
#classifier.add(Flatten())
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 45s 23ms/step - loss: 1.5863 - accuracy: 0.4209
Epoch 2/100
500/500 [==============================] - 12s 23ms/step - loss: 1.0241 - accuracy: 0.6348
Epoch 3/100
500/500 [==============================] - 12s 23ms/step - loss: 0.8128 - accuracy: 0.7128
Epoch 4/100
500/500 [==============================] - 12s 24ms/step - loss: 0.6691 - accuracy: 0.7667
Epoch 5/100
500/500 [==============================] - 12s 24ms/step - loss: 0.5627 - accuracy: 0.8046
Epoch 6/100
500/500 [==============================] - 12s 24ms/step - loss: 0.4808 - accuracy: 0.8348
Epoch 7/100
500/500 [==============================] - 12s 24ms/step - loss: 0.4168 - accuracy: 0.8567
Epoch 8/100
500/500 [==============================] - 12s 24ms/step - loss: 0.3502 - accuracy: 0.8795
Epoch 9/100
500/500 [==============================] - 12s 24ms/step - loss: 0.2980 - accuracy: 0.8988
Epoch 10/100
500/500 [==============================] - 12s 24ms/step - l

In [6]:
#預測test data結果
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.348757028579712
Test accuracy: 0.805899977684021


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.0115324e-03, 1.8121418e-05, 1.0442817e-01, 1.2968809e-02,
        8.7652749e-01, 2.5466858e-07, 9.9008309e-04, 7.9634656e-06,
        4.0475712e-03, 9.3497931e-08]], dtype=float32)